__Problem 1:__ (40 points)
Using the 51 [Electoral College](https://www.archives.gov/electoral-college/2000) numbers that were used in the 2000 USA Presidential Election, estimate the number of ways there could have been a tie in the 2000 Presidential Election.  Use Monte Carlo simulation (using *iid* Bernoulli (1/2); e.g., a fair coin flip for each state) to simulate many elections. The fraction of elections that end in tie multiplied by the number of possible outcomes is your estimate. The exact answer to this question was determined in 2009 by K. Sigman and O. Watanabe to be 17,150,271,124,366. Your estimate should be fairly close to that number. 


Repeat using the [EC numbers](https://www.archives.gov/electoral-college/2020) from the 2020 election. That is create a new list of EC values and feed it to your function.

In [5]:
import random

#list of the number of electoral college votes each state had in the 2000 presidential election
v_2000 = [9,3,8,6,54,8,8,3,3,25,13,4,4,22,12,7,6,8,9,4,10,12,18,10,7,11,3,5,4,4,15,5,33,
          14,3,21,8,7,23,4,8,3,11,32,5,3,13,11,5,11,3]

#list of the number of electoral college votes each state had in the 2020 presidential election
v_2020 = [9,3,11,6,55,9,7,3,3,29,16,4,4,20,11,6,6,8,8,4,10,11,16,10,6,10,3,5,6,4,14,5,29,
          15,3,18,7,7,20,4,9,3,11,38,6,3,13,12,5,10,3]

# create the function below to return the number of ways
# a particular Electoral College outcome may happen
# the parameters are as follows:
# ec: this is the list of EC values to be used in your simulation
# target: this is the number of EC values candidate 1 should win
# trials: this is the number of trials you MC simulation should use
def target_estimator(ec,target,trials):
    success=0
    for i in range(trials):
        outcomes=[random.random()<.5 for i in range(len(ec))]
        ec_outcomes=[ec[i] if outcomes[i] else 0 for i in range(len(outcomes))]
        if sum(ec_outcomes)==target:
            success=success+1
    
    return (success/trials)*(2**51)
    
# now we use the function to determine the number of ways a tie can occur
# which is equivalent to candidate 1 winning 269 votes
result_2000 = target_estimator(v_2000,269,1000000)
result_2020 = target_estimator(v_2020,269,1000000)

print('result 2000: ',result_2000)
print('result 2020: ',result_2020)

result 2000:  17464959354942.783
result 2020:  17039369190156.271


__Problem 2:__ In the year 2000 (Bush versus Gore), the situation right before the election was this: Bush had (in his pocket) 24 states totaling 210 EC votes, while Gore had 10 states totaling 146 EC votes. There were 17 states left over, the “Battleground States”, in which it was supposedly unclear who would win them. Look at the file [2000.pdf](http://www.cs.columbia.edu/~cannon/2000.pdf) to see exactly what states made up the 17, and the EC numbers for them.

__Part I:__ (30 points) First assume that each Battleground State outcome is determined by an *iid* fair coin toss; Bernoulli (1/2). Simulate (using 1 million copies to average, using Monte Carlo) to obtain the probability that Bush would win the election, and the probability that Gore would win the election, and the probability of a tie.

In [22]:
#list containing the number of electoral college votes each of 17 battleground states had in 2000 election
v_in_play= [6,3,25,22,7,4,18,10,11,4,4,7,23,11,11,5,11]

# this function returns an estimate for the probability that candidate 1
# wins in a US Presidential election given that they already have
# ec_in_the_bag EC votes.
# v_left is a list of the remaining EC numbers,
# trials is the number of trials to be used in the MC simulation.
# This function assumes that the probability of winning any remaining state
# is 0.5

def ec_estimator(ec_in_the_bag,v_left,trials):
    count=0
    for i in range(trials):
        outcomes=[random.random()<.5 for i in range(len(v_left))]
        ec_outcomes=[v_left[i] if outcomes[i] else 0 for i in range(len(outcomes))]
        if sum(ec_outcomes)>269-(ec_in_the_bag):
            count=count+1
  
    return count/trials
        

# estimate for Bush win
print('Bush win: ',ec_estimator(210,v_in_play,1000000))

# estimate for Gore win
print('Gore win: ',ec_estimator(146,v_in_play,1000000))

#estimate for Tie
print('tie: ',target_estimator(v_in_play,269-146,1000000)/2**51)

#check that it's the same (close) with
print('tie: ',target_estimator(v_in_play,269-210,1000000)/2**51)



Bush win:  0.879186
Gore win:  0.112813
tie:  0.007582
tie:  0.007527


 __Part II:__ (30 points) In the [2000.pdf](http://www.cs.columbia.edu/~cannon/2000.pdf) file, you will also see the probabilities that had been determined by extensive polling for Gore winning each of the 17 states. Denote these probabilities by $p_1,...,p_{17}$. No longer are they all *p = 1/2* as we assumed in Part I. For example, for the state of Wisconsin (WI), *p = 0.946*, while for the state of Nevada (NV), p = 0.146. Only for the state of Maine (ME) is p = 0.5. Now re-do the simulation in Part I using the 17 Bernoulli $(p_i)$. The idea now is that each of the 17 states has its own coin so to speak.

## Explanation of Design and Style Choices

First, I created a parameterized function because doing so allows the function to be re-used (as opposed to hard-coding it) in the future with different parameters. It makes for a more flexible, functional, and universal problem-solving strategy.

Second, I used a list of *tuples* for the function because tuples are immutable. This makes them run faster than mutable datatypes, especially at large scales. Additionally, tuples make the function "safer"/more robust in that it does not rely on the unity of multiple parallel lists (that might accidentally get changed at some point in the process) to operate properly. 

Finally, I found the probability of a tie via subtraction at the end because ultimately I thought it was simpler than attempting to calculate a tie within the function itself, which would either require new, unique parameters or multiple extra steps within the function itself.

In [26]:
#each tuple has:
#1.Gore's probability of winning a battleground state 
#2.that state's number of electoral college votes
gore_tuple=[(.395,6),(.143,3),(.893,25),(.999,22),(.420,7),(.500,4),(.997,18),
               (.999, 10),(.236,11),(.146,4),(.731,4),(.602,7),
               (.989,23),(.289,11),(.753,11),(.302,5),(.946,11)]

#each tuple has:
#1.Bush's probability of winning a battleground state 
#2.that state's number of electoral college votes
bush_tuple=[(.605,6),(.857,3),(.107,25),(.001,22),(.58,7),(.5,4),(.003,18),
               (.001, 10),(.764,11),(.854,4),(.269,4),(.398,7),
               (.011,23),(.711,11),(.247,11),(.698,5),(.054,11)]


#parameters are:
#1. number of electoral college votes a candidate already has
#2. list of tuples with candidate's probability of winning a state and that state's EC votes
#3. number of trials (number of times you want to run the function)
def precise_ec_estimator(ec_in_the_bag,election_tuple,trials):
    #count keeps track of how many time a candidate wins/gets EC votes over 270
    count=0
    
    #for loop for number of trials given
    for j in range(trials):
        #creates list of T/F values using randomly generated numbers 
        #list represents whether or not candidate wins a state
        outcomes=[random.random()<election_tuple[i][0] for i in range(len(election_tuple))]
        #using list of T/F values, create new list based off of EC values of each state won
        ec_outcomes=[election_tuple[i][1] if outcomes[i] else 0 for i in range(len(outcomes))]
        #sum the amount of EC values won, check if puts a candidate over 270 votes
        if sum(ec_outcomes)>(269-(ec_in_the_bag)):
            count=count+1

    #finds average amount of times a candidate won 
    return (count/trials)


# estimate for probability of Gore win
print('Gore win: ', precise_ec_estimator(146,gore_tuple,1000000))

# estimate for probability of Bush win
print('Bush win: ', precise_ec_estimator(210,bush_tuple,1000000))

#estimate for tie by subtracting probability of Bush win and Gore win from 1 
#any amount left over after subtraction is probability of tie
prob_tie=1-precise_ec_estimator(210,bush_tuple,1000000)-precise_ec_estimator(146,gore_tuple,1000000)
print("Tie:","%.5f" % prob_tie)

Gore win:  0.846993
Bush win:  0.14218
Tie: 0.01134
